In [4]:
from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


In [ ]:
!pip install SoccerNet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.1 MB/s eta 0:00:00


In [ ]:
from SoccerNet.Downloader import SoccerNetDownloader
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory="/content/drive/MyDrive/Colab Notebooks/Artificial Vision/Contest/SoccerNet")
#scarichiamo lo split di train per finetunare il modello sui nostri dati
mySoccerNetDownloader.downloadDataTask(task="tracking-2023", split=["train"])

In [5]:
import os
import zipfile
from tqdm.notebook import tqdm

# Percorsi
zipFile_path = "/content/drive/MyDrive/Colab Notebooks/Artificial Vision/Contest/SoccerNet/tracking-2023/train.zip"
train_data = "/content/train"
local_zip = "/content/train_dataset.zip"

# Funzione per copiare con progress bar
def copy_with_progress(src, dst):
    total_size = os.path.getsize(src)
    with open(src, 'rb') as fsrc, open(dst, 'wb') as fdst:
        with tqdm(total=total_size, unit='B', unit_scale=True, desc="Copia da Drive") as pbar:
            while True:
                # Leggiamo a blocchi di 1MB per efficienza
                chunk = fsrc.read(1024 * 1024)
                if not chunk:
                    break
                fdst.write(chunk)
                pbar.update(len(chunk))

# Logica principale
if not os.path.exists(train_data):
    print("Inizio procedura di estrazione...")

    # 1. Copia da Drive
    if not os.path.exists(local_zip): # Copia solo se non c'è già lo zip locale
        copy_with_progress(zipFile_path, local_zip)
    else:
        print("Zip locale già presente, salto la copia.")

    # 2. Unzip (con barra)
    print("Analisi contenuto zip...")
    with zipfile.ZipFile(local_zip, 'r') as zf:
        # Recuperiamo la lista dei file per impostare la barra
        file_list = zf.infolist()

        # Estraiamo un file alla volta aggiornando la barra
        for member in tqdm(file_list, desc="Estrazione file", unit="file"):
            zf.extract(member, train_data)

    # 3. Pulizia
    os.remove(local_zip)
    print(f"Finito! Dataset pronto in: {train_data}")

else:
    print("Dati già presenti nella cartella di destinazione!")

Inizio procedura di estrazione...


Copia da Drive:   0%|          | 0.00/9.58G [00:00<?, ?B/s]

Analisi contenuto zip...


Estrazione file:   0%|          | 0/43207 [00:00<?, ?file/s]

Finito! Dataset pronto in: /content/train


In [6]:
# Sposta tutto il contenuto di train/train in train/
!mv /content/train/train/* /content/train/

# Rimuovi la cartella vuota interna
!rmdir /content/train/train

# Verifica
!ls /content/train | head

SNMOT-060
SNMOT-061
SNMOT-062
SNMOT-063
SNMOT-064
SNMOT-065
SNMOT-066
SNMOT-067
SNMOT-068
SNMOT-069


In [7]:
import re

# Configurazione Percorso
TRAIN_FOLDER_PATH = "/content/train"

def get_ball_ids_from_gameinfo(gameinfo_file):
    ball_ids = set()
    try:
        with open(gameinfo_file, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line or "=" not in line:
                    continue

                left, right = line.split("=", 1)
                left = left.strip()
                right = right.strip()

                label = right.split(";", 1)[0].strip().lower()
                if label != "ball":
                    continue

                m = re.match(r"trackletID_(\d+)$", left, flags=re.IGNORECASE)
                if m:
                    ball_ids.add(m.group(1))
    except FileNotFoundError:
        return set()

    return ball_ids

def filter_gt_in_place(video_folder_path, dir_name):
    gameinfo_path = os.path.join(video_folder_path, "gameinfo.ini")
    gt_file_path = os.path.join(video_folder_path, "gt", "gt.txt")

    if not os.path.exists(gameinfo_path) or not os.path.exists(gt_file_path):
        return False

    ball_ids = get_ball_ids_from_gameinfo(gameinfo_path)

    if not ball_ids:
        return False

    try:
        with open(gt_file_path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        filtered_lines = []
        removed_count = 0

        for line in lines:
            parts = line.strip().split(",")
            if len(parts) < 2:
                filtered_lines.append(line)
                continue

            obj_id = parts[1].strip()

            if obj_id in ball_ids:
                removed_count += 1
                continue

            filtered_lines.append(line)

        if removed_count > 0:
            with open(gt_file_path, "w", encoding="utf-8") as f:
                f.writelines(filtered_lines)

            tqdm.write(f"[OK] {dir_name} | ball_ids={sorted(list(ball_ids))} | righe rimosse={removed_count}")

            return True

        return False

    except Exception as e:
        tqdm.write(f"[ERR] Errore processando {gt_file_path}: {e}")
        return False

def process_train_dataset():
    print(f"Inizio rimozione 'ball' dai GT in: {TRAIN_FOLDER_PATH}")

    if not os.path.exists(TRAIN_FOLDER_PATH):
        print(f"Errore: La cartella {TRAIN_FOLDER_PATH} non esiste.")
        return

    dirs = [d for d in os.listdir(TRAIN_FOLDER_PATH) if os.path.isdir(os.path.join(TRAIN_FOLDER_PATH, d))]
    dirs.sort()

    modified_files = 0

    for dir_name in tqdm(dirs, desc="Pulizia Dataset", unit="video"):
        video_folder_path = os.path.join(TRAIN_FOLDER_PATH, dir_name)

        if filter_gt_in_place(video_folder_path, dir_name):
            modified_files += 1

    print(f"\n[COMPLETATO]")
    print(f"Totale video analizzati: {len(dirs)}")
    print(f"File gt.txt modificati: {modified_files}")

process_train_dataset()

Inizio rimozione 'ball' dai GT in: /content/train


Pulizia Dataset:   0%|          | 0/57 [00:00<?, ?video/s]

[OK] SNMOT-060 | ball_ids=['18'] | righe rimosse=734
[OK] SNMOT-061 | ball_ids=['1', '27'] | righe rimosse=740
[OK] SNMOT-062 | ball_ids=['19'] | righe rimosse=750
[OK] SNMOT-063 | ball_ids=['19'] | righe rimosse=750
[OK] SNMOT-064 | ball_ids=['23'] | righe rimosse=590
[OK] SNMOT-065 | ball_ids=['24', '26'] | righe rimosse=675
[OK] SNMOT-066 | ball_ids=['24'] | righe rimosse=724
[OK] SNMOT-067 | ball_ids=['22'] | righe rimosse=707
[OK] SNMOT-068 | ball_ids=['8'] | righe rimosse=718
[OK] SNMOT-069 | ball_ids=['18', '24'] | righe rimosse=768
[OK] SNMOT-070 | ball_ids=['12'] | righe rimosse=737
[OK] SNMOT-071 | ball_ids=['20'] | righe rimosse=701
[OK] SNMOT-072 | ball_ids=['1'] | righe rimosse=650
[OK] SNMOT-073 | ball_ids=['19'] | righe rimosse=711
[OK] SNMOT-074 | ball_ids=['23'] | righe rimosse=750
[OK] SNMOT-075 | ball_ids=['22', '24'] | righe rimosse=552
[OK] SNMOT-076 | ball_ids=['25'] | righe rimosse=609
[OK] SNMOT-077 | ball_ids=['20'] | righe rimosse=733
[OK] SNMOT-097 | ball_ids

In [ ]:
import os, glob, random
import cv2
from tqdm.notebook import tqdm

TRAIN_FOLDER_PATH = "/content/train"
OUT_ROOT = "/content/drive/MyDrive/Colab Notebooks/Artificial Vision/Contest/reid_dataset"

VAL_FRACTION_VIDEOS = 0.2
SEED = 42

PAD_FRAC = 0.15
MIN_H = 60
MIN_W = 20
BLUR_LAPLACIAN_THR = 18.0
MAX_PER_ID = 120

def list_frames(img1_dir):
    files = glob.glob(os.path.join(img1_dir, "*.jpg")) + glob.glob(os.path.join(img1_dir, "*.png"))
    return sorted(files)

def read_mot(gt_path):
    out=[]
    with open(gt_path,"r") as f:
        for line in f:
            line=line.strip()
            if not line:
                continue
            p=line.split(",")
            if len(p)<6:
                continue
            fr=int(float(p[0])); tid=str(int(float(p[1])))
            x=float(p[2]); y=float(p[3]); w=float(p[4]); h=float(p[5])
            out.append((fr,tid,x,y,w,h))
    return out

def crop_pad(img, x,y,w,h, pad_frac):
    H,W = img.shape[:2]
    padw=w*pad_frac; padh=h*pad_frac
    x1=int(round(x-padw)); y1=int(round(y-padh))
    x2=int(round(x+w+padw)); y2=int(round(y+h+padh))
    x1=max(0, min(W-1, x1)); y1=max(0, min(H-1, y1))
    x2=max(1, min(W,   x2)); y2=max(1, min(H,   y2))
    if x2<=x1 or y2<=y1:
        return None
    return img[y1:y2, x1:x2].copy()

def blur_score(crop):
    g=cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(g, cv2.CV_64F).var()

def extract_reid(TRAIN_FOLDER_PATH, OUT_ROOT):
    random.seed(SEED)
    os.makedirs(OUT_ROOT, exist_ok=True)
    train_root=os.path.join(OUT_ROOT,"train")
    val_root=os.path.join(OUT_ROOT,"val")
    os.makedirs(train_root, exist_ok=True)
    os.makedirs(val_root, exist_ok=True)

    seqs = sorted([d for d in os.listdir(TRAIN_FOLDER_PATH) if os.path.isdir(os.path.join(TRAIN_FOLDER_PATH,d))])
    random.shuffle(seqs)
    n_val = max(1, int(round(len(seqs)*VAL_FRACTION_VIDEOS)))
    val_set = set(seqs[:n_val])

    per_id_count={}
    saved=0

    for seq in tqdm(seqs, desc="Extract ReID", unit="seq"):
        seq_dir = os.path.join(TRAIN_FOLDER_PATH, seq)
        img1 = os.path.join(seq_dir, "img1")
        gt   = os.path.join(seq_dir, "gt", "gt.txt")
        if not os.path.isdir(img1) or not os.path.isfile(gt):
            continue

        frames = list_frames(img1)
        if not frames:
            continue

        # mapping frame->path (1-based)
        frame_to_path = {i:p for i,p in enumerate(frames, start=1)}
        lines = read_mot(gt)

        split_root = val_root if seq in val_set else train_root

        by_frame={}
        for fr,tid,x,y,w,h in lines:
            by_frame.setdefault(fr,[]).append((tid,x,y,w,h))

        for fr, dets in by_frame.items():
            ip = frame_to_path.get(fr)
            if ip is None:
                continue
            img = cv2.imread(ip)
            if img is None:
                continue

            for tid,x,y,w,h in dets:
                if h<MIN_H or w<MIN_W:
                    continue
                crop = crop_pad(img,x,y,w,h,PAD_FRAC)
                if crop is None:
                    continue
                if blur_score(crop) < BLUR_LAPLACIAN_THR:
                    continue

                identity = f"{seq}_{tid}"
                per_id_count.setdefault(identity,0)
                if per_id_count[identity] >= MAX_PER_ID:
                    continue

                od = os.path.join(split_root, identity)
                os.makedirs(od, exist_ok=True)
                out_name = f"{seq}_f{fr:06d}_id{int(tid):04d}_{per_id_count[identity]:04d}.jpg"
                cv2.imwrite(os.path.join(od,out_name), crop)

                per_id_count[identity]+=1
                saved+=1

    print("[DONE] saved:", saved, "| identities:", len(per_id_count))
    return saved

saved = extract_reid(TRAIN_FOLDER_PATH, OUT_ROOT)


Extract ReID:   0%|          | 0/57 [00:00<?, ?seq/s]

KeyboardInterrupt: 

In [9]:
import glob, os
train_imgs = glob.glob(f"{OUT_ROOT}/train/*/*.jpg")
val_imgs   = glob.glob(f"{OUT_ROOT}/val/*/*.jpg")
print("train images:", len(train_imgs))
print("val images:", len(val_imgs))
print("train IDs:", len(glob.glob(f'{OUT_ROOT}/train/*')))
print("val IDs:", len(glob.glob(f'{OUT_ROOT}/val/*')))


NameError: name 'OUT_ROOT' is not defined

In [2]:
!pip -q install torchreid


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [12]:
import os, glob, random
from collections import defaultdict

OUT_ROOT = "/content/drive/MyDrive/Colab Notebooks/Artificial Vision/Contest/reid_dataset"
TRAIN_DIR = f"{OUT_ROOT}/train"
VAL_OOD_DIR = f"{OUT_ROOT}/val"

SEED = 42
VAL_INTRA_FRAC = 0.1
MAX_PER_ID_TRAIN = None

random.seed(SEED)

def build_intra_split(train_dir):
    id_folders = sorted([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir,d))])
    train_samples, val_samples = [], []
    id2label = {pid:i for i,pid in enumerate(id_folders)}

    for pid in id_folders:
        files = sorted(glob.glob(os.path.join(train_dir, pid, "*.jpg")))
        random.shuffle(files)
        if MAX_PER_ID_TRAIN is not None:
            files = files[:MAX_PER_ID_TRAIN]

        n_val = max(1, int(len(files)*VAL_INTRA_FRAC))
        val_files = files[:n_val]
        tr_files  = files[n_val:]

        y = id2label[pid]
        val_samples += [(f,y) for f in val_files]
        train_samples += [(f,y) for f in tr_files]

    return train_samples, val_samples, len(id_folders)

train_samples, val_intra_samples, num_classes = build_intra_split(TRAIN_DIR)
print("num_classes:", num_classes)
print("train samples:", len(train_samples))
print("val_intra samples:", len(val_intra_samples))


num_classes: 368
train samples: 37589
val_intra samples: 4171


In [16]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image
import torchreid
from tqdm.notebook import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)
if DEVICE == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

class ReidList(Dataset):
    def __init__(self, samples, tfm):
        self.samples = samples
        self.tfm = tfm
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, i):
        path, y = self.samples[i]
        img = Image.open(path).convert("RGB")
        img = self.tfm(img)
        return img, y

train_tfm = T.Compose([
    T.Resize((256,128)),
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.10, hue=0.02),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

val_tfm = T.Compose([
    T.Resize((256,128)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])


BATCH = 256


NUM_WORKERS = 4
PIN = (DEVICE == "cuda")

train_ds = ReidList(train_samples, train_tfm)
val_ds   = ReidList(val_intra_samples, val_tfm)

train_dl = DataLoader(
    train_ds,
    batch_size=BATCH,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=PIN,
    persistent_workers=(NUM_WORKERS > 0),
    prefetch_factor=2
)

val_dl = DataLoader(
    val_ds,
    batch_size=BATCH,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN,
    persistent_workers=(NUM_WORKERS > 0),
    prefetch_factor=2
)

DEVICE: cuda
GPU: Tesla T4


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
model = torchreid.models.build_model(
    name="osnet_x0_25",
    num_classes=num_classes,
    pretrained=True
).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=1e-4)

scaler = torch.amp.GradScaler('cuda', enabled=(DEVICE == "cuda"))

@torch.no_grad()
def eval_acc():
    model.eval()
    correct = 0
    total = 0

    for x, y in val_dl:
        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE, non_blocking=True)

        with torch.amp.autocast('cuda', enabled=(DEVICE == "cuda")):
            feats = model(x)
            logits = model.classifier(feats)

        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        total += y.numel()

    return correct / total if total else 0.0

EPOCHS = 10
best = 0.0
best_path = f"{OUT_ROOT}/osnet_x0_25_finetuned_full.pth"

for ep in range(1, EPOCHS+1):
    model.train()
    pbar = tqdm(train_dl, desc=f"Epoch {ep}/{EPOCHS}")
    for x, y in pbar:
        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast('cuda', enabled=(DEVICE == "cuda")):
            logits = model(x)           # in train ritorna logits 
            loss = criterion(logits, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        pbar.set_postfix(loss=loss.detach().item())

    acc = eval_acc()
    print(f"epoch {ep:02d} | val_intra_acc={acc:.4f}")

    if acc > best:
        best = acc
        torch.save(model.state_dict(), best_path)

print("BEST val_intra_acc:", best)
print("Saved:", best_path)



DEVICE: cuda
GPU: Tesla T4
Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x0_25_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1/10:   0%|          | 0/147 [00:00<?, ?it/s]

epoch 01 | val_intra_acc=0.0012


Epoch 2/10:   0%|          | 0/147 [00:00<?, ?it/s]

...

epoch 08 | val_intra_acc=0.0017


Epoch 9/10:   0%|          | 0/147 [00:00<?, ?it/s]

epoch 09 | val_intra_acc=0.0012


Epoch 10/10:   0%|          | 0/147 [00:00<?, ?it/s]

epoch 10 | val_intra_acc=0.0014
BEST val_intra_acc: 0.002397506593143131
Saved: /content/drive/MyDrive/Colab Notebooks/Artificial Vision/Contest/reid_dataset/osnet_x0_25_finetuned_full.pth


In [ ]:
import torch

in_path  = f"{OUT_ROOT}/osnet_x0_25_finetuned_full.pth"
out_path = f"{OUT_ROOT}/osnet_x0_25_finetuned_backbone_only.pth"

sd = torch.load(in_path, map_location="cpu")
sd2 = {k:v for k,v in sd.items() if "classifier" not in k}

torch.save(sd2, out_path)
print("Saved:", out_path)

#osnet_x0_25_finetuned_full.pth = backbone + classifier (testa di classificazione)
#osnet_x0_25_finetuned_backbone_only.pth = solo backbone, perfetto per estrarre embedding

NameError: name 'OUT_ROOT' is not defined